The conversion of displacements into cartesians nessecitates the use of an iterative procedure called the back transformation.
As with the conversion of forces or gradients, implementation of the pseudo inverse leads to the similar looking result.
$$ \textbf G^{-1}_0 \textbf {B}^T_0 u\Delta q_0 = \Delta x_0$$

Here, the G matrix has dimensions of cartesian not internals as it is calculated with $\textbf B^Tu \textbf B$ where the u matrix has dimesnions of the number of internal coordinates

The iterative back transformation becomes nessecary because the new cartesian geometry is not perfectly eqivalent to the new geometry we stepped to in internals
$$x_0 + \Delta x = x_1$$
To converge on the correct cartesian geometry, we take the error in internals and convert again to cartesians

Call the cartesian geometry converted back to internals as previously shown, $q_x$ and the geometry we calcualted $q_1$ then
$$q_x - q_1 = q_\epsilon$$
We can then resolve our convesian equation having recalulated the nessecary matrices at the new geometry
$$ \textbf G^{-1}_n \textbf {B}_n^T u\Delta q_\epsilon = \Delta x_n$$

To demonsrtrate the need for the back transformation, we'll take a step, in internal coordinates, of 0.2 bohr for the bond lengths and then convert it into cartesians, and back again into internals

In [ ]:
# A simple Psi4 input script to compute a Wilson B matrix
# edited with jupyter
# Created by: Rollin A. King
# Date: 7/27/17
# License: GPL v3.0

import numpy as np
np.set_printoptions(precision=5, linewidth=200, suppress=True)
import psi4

# Memory for Psi4 in GB
psi4.set_memory('1 GB')
psi4.core.set_output_file('output.dat',False)

import optking
# Memory for numpy in GB
numpy_memory = 2

mol = psi4.geometry("""
O
H 1 1.1
H 1 1.1 2 104
symmetry c1
""")

# Set some options
psi4.set_options({"basis": "cc-pvdz",
                  "scf_type": "pk",
                  "e_convergence": 1e-8})

In [ ]:
mol.update_geometry()
xyz = np.array( mol.geometry() )
print('Cartesian geometry')
print(xyz)

#from optking import printTools
optking.printInit(psi4.core.print_out)

# Create some internal coordinates for water.
# Numbering starts with atom 0.
from optking import stre,bend
r1 = stre.STRE(0,1)
r2 = stre.STRE(0,2)
theta = bend.BEND(1,0,2)
intcos = [r1,r2,theta]

print("%15s%15s" % ('Coordinate', 'Value'))
for I in intcos:
  print("%15s = %15.8f %15.8f" % (I, I.q(xyz), I.qShow(xyz)))


from optking import optParams as op
userOptions = {'print' : 2}
op.Params = op.OPT_PARAMS(userOptions)
optking.printTools.print_opt("\tParameters from optking.optimize\n")
optking.printTools.print_opt( str(op.Params) )

# displacements to be made in au
dq = np.array( [0.0, 0.2, 0.0], float)

# forces - only used for printing here
fq = np.array( [0.0, 0.0, 0.0], float)

from optking import displace
displace.displace(intcos, xyz, dq, fq)

print('Final Displaced Cartesian geometry')
print(xyz)

# Check result
print("%15s%15s" % ('Coordinate', 'Value'))
for I in intcos:
  print("%15s = %15.8f %15.8f" % (I, I.q(xyz), I.qShow(xyz)))

In [ ]:
from optking import intcosMisc
from optking import linearAlgebra
dq0 = np.zeros(len(intcos), float)
for i in range (4):
    dq0[i] = 0.2

#This is what we should be stepping to    
qGeomWanted = np.add(qGeom, dq0)
print("New internal geometry")
printTools.printArray(qGeomWanted)

B0 = intcosMisc.Bmat(intcos, xyzGeom)

G0 = np.dot(B0.T, B0) 
#the u identity matrix is omittted above due to the dimensions
printTools.printMat(G0)    
G0inv = linearAlgebra.symmMatInv(G0, True)
u = np.identity(len(intcos))
dx0 = np.dot(G0inv, np.dot(B0.T, np.dot(u, dq0)))

print ("Displacement in cartesians")
printTools.printArray(dx0)
dx0mat = np.reshape(dx0, (5, 3))
#Compare the cartesian geometries
xyzGeom0 = np.add(dx0mat, xyzGeom)
print ("Old geometry")
print (xyzGeom)
print ("New geometry")
print(xyzGeom0)


qGeom1 = np.zeros(len(intcos), float)
for i in range (len(intcos)):
    qGeom1[i] = (intcos[i].q(xyzGeom0))
print ("New geometry with error")    
print (qGeom1)
dq1Prime = np.subtract(qGeom1, qGeom0) #This is the actual Dq we took
print ("Acutal step from cartesians")
print (dq1Prime)

Dq1 = np.subtract(dq0, dq1Prime) #This is the error in Dq

print ("This is why we need to back transformation")
print (Dq1)